## sub-task 1 제출 스크립트
ETRI FASHION-HOW Season 4 task1 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: test.py)를 본 스크립트 내의 etri_task1_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task1_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory==1.8.7
!pip install imblearn

### 2. etri_task1_submit() 함수 편집 (추론 스크립트)
#### etri_task1_submit() 편집 시 주의사항

1. 아래 etri_task1_submit() 함수 내에 전체 추론 실행 코드를 삽입하고 결과를 dataframe으로 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로)
5. 데이터는 /aif/Dataset/ 경로 아래에 있습니다. (코드 내용 참조) 
6. return할 결과물과 양식에 유의합니다.

In [2]:
def etri_task1_submit():
    
    # from utils.dataset_add import test_, test_tta
    from utils.dataset_one import test_, test_tta
    # from add_res_net import Resnet, ResNetResidualConnectionModel
    from res_net import Resnet, ResNetResidualConnectionModel, ResNetFeedForwardModel

    import pandas as pd
    import numpy as np

    import torch
    import torch.utils.data
    import torch.utils.data.distributed
    from tqdm import tqdm

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ResNetFeedForwardModel().to(device)
    model.eval()
    trained_weights = torch.load('./models/best/loss_best', map_location=device) # 자기 모델 경로를 지정합니다
    model.load_state_dict(trained_weights)
    df = pd.read_csv('/aif/Dataset/info_etri20_emotion_test.csv') # 제출 시 데이터 경로 준수. /aif/ 아래에 있습니다.
    daily_pred_list = np.array([])
    gender_pred_list = np.array([])
    embel_pred_list = np.array([])
#-------------------------------------- one --------------------------------------

    valid_dataset = test_tta(df)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=0)

    # with torch.no_grad():
    #     for images, d_labels, g_labels, e_labels in tqdm(iter(valid_loader)):
    #         images, d_labels, g_labels, e_labels = images.to(device), d_labels.to(device), g_labels.to(device), e_labels.to(device)
    #         d_outputs, g_outputs, e_outputs = model(images)

    #         daily_indx = torch.argmax(d_outputs, dim=1)
    #         gender_indx = torch.argmax(g_outputs, dim=1)
    #         embel_indx = torch.argmax(e_outputs, dim=1)

    #         daily_pred_list = np.concatenate([daily_pred_list, daily_indx.cpu()], axis=0)
    #         gender_pred_list = np.concatenate([gender_pred_list, gender_indx.cpu()], axis=0)
    #         embel_pred_list = np.concatenate([embel_pred_list, embel_indx.cpu()], axis=0)

    
    with torch.no_grad():
        for images, d_labels, g_labels, e_labels in tqdm(iter(valid_loader)):
            d_labels, g_labels, e_labels = d_labels.to(device), g_labels.to(device), e_labels.to(device)

            d_output = []
            g_output = []
            e_output = []

            for img in images:
                img = img.to(device)
                d_out, g_out, e_out = model(img)
                d_output.append(d_out)
                g_output.append(g_out)
                e_output.append(e_out)

            d_outputs = sum(d_output) / len(d_output)
            g_outputs = sum(g_output) / len(g_output)
            e_outputs = sum(e_output) / len(e_output)

            daily_indx = torch.argmax(d_outputs, dim=1)
            gender_indx = torch.argmax(g_outputs, dim=1)
            embel_indx = torch.argmax(e_outputs, dim=1)

            daily_pred_list = np.concatenate([daily_pred_list, daily_indx.cpu()], axis=0)
            gender_pred_list = np.concatenate([gender_pred_list, gender_indx.cpu()], axis=0)
            embel_pred_list = np.concatenate([embel_pred_list, embel_indx.cpu()], axis=0)

#-------------------------------------- three --------------------------------------

    # valid_dataset1, valid_dataset2, valid_dataset3 = test_(df)
    # valid_loader1 = torch.utils.data.DataLoader(valid_dataset1, batch_size=8, shuffle=False, num_workers=0)
    # valid_loader2 = torch.utils.data.DataLoader(valid_dataset2, batch_size=8, shuffle=False, num_workers=0)
    # valid_loader3 = torch.utils.data.DataLoader(valid_dataset3, batch_size=8, shuffle=False, num_workers=0)

    # with torch.no_grad():
    #     for batch1, batch2, batch3 in tqdm(zip(valid_loader1, valid_loader2, valid_loader3)):

# #--------------------------------- tta -------------------------------
    #         # d_output = []
    #         # g_output = []
    #         # e_output = []
    #         # images1, _ = batch1
    #         # images2, _ = batch2
    #         # images3, _ = batch3
            
    #         # for img in images1:
    #         #     img = img.to(device)
    #         #     d_output.append(model(img,'d'))
            
    #         # for img in images2:
    #         #     img = img.to(device)
    #         #     g_output.append(model(img,'g'))
            
    #         # for img in images3:
    #         #     img = img.to(device)
    #         #     e_output.append(model(img,'e'))

    #         # d_outputs = sum(d_output) / len(d_output)
    #         # g_outputs = sum(g_output) / len(g_output)
    #         # e_outputs = sum(e_output) / len(e_output)

    #         # #--------------------------------- three -------------------------------

            # images1, _ = batch1
            # images2, _ = batch2
            # images3, _ = batch3
            
            # images1 = images1.to(device)
            # images2 = images2.to(device)
            # images3 = images3.to(device)
            
            # d_outputs = model(images1,'d')
            # g_outputs = model(images2,'g')
            # e_outputs = model(images3,'e')

            # daily_pred = d_outputs
            # daily_indx = torch.argmax(daily_pred, dim=1)
            # daily_pred_list = np.concatenate([daily_pred_list, daily_indx.cpu()], axis=0)

            # gender_pred = g_outputs
            # gender_indx = torch.argmax(gender_pred, dim=1)
            # gender_pred_list = np.concatenate([gender_pred_list, gender_indx.cpu()], axis=0)

            # embel_pred = e_outputs
            # embel_indx = torch.argmax(embel_pred, dim=1)
            # embel_pred_list = np.concatenate([embel_pred_list, embel_indx.cpu()], axis=0)
            

        # 예측 결과를 dataframe으로 변환한 다음 함수의 결과로 return합니다.
        # 'image_name', 'daily', 'gender', 'embel'의 컬럼명과 image_name의 샘플 순서를 지켜주시기 바랍니다.
        # Baseline이 아닌 다른 모델을 사용하는 경우에도 같은 형식의 dataframe으로 return할 수 있도록 합니다.


    out = pd.DataFrame({'image_name':df['image_name'], 'daily':daily_pred_list, 'gender':gender_pred_list, 'embel':embel_pred_list})
   
    return out # 반드시 추론결과를 return

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task1_submit 함수를 wrapping해야 합니다.

In [3]:
def submit():
    return etri_task1_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [4]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="task1662",             # 본인의 모델명 입력(버전 관리에 용이하게끔 편의에 맞게 지정합니다)
               key="8c393154-3345-44fc-9ebe-de0c97bdcfb1",                        # 본인의 task key 입력
               func=submit                                 # 3.에서 wrapping한 submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : task.py
python
파일 전송 완료
error = [Errno 2] No such file or directory: './models/best/loss_best'
106.95641994476318
